# Far-field, near-field, and source level

(acoustics-near_field)=
## The Near-field of a Transducer

One important property of the far-field of a transducer is that the sound in this region all appears to coming from a single point on the transducer. This is what leads to the  $1/r^2$ decrease in the intensity with range. This is no longer true near the face of the transducer. There is a region where the sound from one part of the transducer is interfering with sound from other parts of the transducer and there are no clear beams. This region is refered to as the **near-field** of the transducer. There is no simple mathematical expression for the sound field in this region, but we can numerically determine the full field for the baffled piston and the result for $f = 12 kHz$ and $a = 0.5 m$ is shown in the plot on the right below. The plot on the left shows the far-field approximation using the same set of paramters. In the far-field calculation plot, regardless of how close you get to the transducer face, the field will always appear to come from a point at the center of the transducer. In the full-field calculation, this is no longer true. There is sound coming from the entire face of the transducer, which spans -0.5 m to 0.5 m, and there is a complicated interference pattern out to about 2 m in range. Beyond this point, the tow fields (far-field and full-field) start to become similar.

```{image} ../images/FarField_NearField_example.png
:width: 400px
```

This transition between the near-field and the far-field of the transducer can be seen more clearly if we look just at the intensity along the center of the main beam. The intensity of the full-field along this axis has an analytic expression. The widget below shows both the full-field intensity (red line) and the far-field approximation (blue line). Notice that as you go further out in range, the two curves begin to converge which indicates when the transition from the near-field to the far-field has occured. There are a number of ways to estimate when this transition between near-field and far-field occurs. One useful estimate is that the range from the transducer needs to satisfy $r \gg 2a$ and $r \gg \pi a^2/\lambda$. In these calculations, the speed of sound is $c = 1500$ m/s and using the relationship between frequency and wavelength, $c = f\lambda$ , you can verify that these conditions hold as you change the frequency and radius of the transducer. 

In [1]:
# Import packages
import multiprocessing as mp
from functools import partial

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import j1  # Bessel function of the first kind
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd

In [2]:
# Constants
rho0 = 1
c = 1500

# Create sliders for 'SL', 'a', 'f'
f_slider = widgets.IntSlider(min=1, max=200, step=1, value=12, description='Frequency (kHz)')
a_slider = widgets.FloatSlider(min=0.01, max=1, step=0.01, value=0.5, description='Piston Radius (m)')
SL_slider = widgets.IntSlider(min=100, max=220, step=1, value=180, description='Source Level (dB)')

def far_and_full_field_intensity_on_axis(f, a, SL):
    # Calculate angular frequency
    omega = 2 * np.pi * f * 1000

    # Calculate wavenumber
    k = omega / c

    # Calculate sound pressure level in linear units
    SL_lin = 10**(SL / 20)

    # Calculate the particle velocity amplitude
    U0 = SL_lin / (rho0 * c * k * a * a / 2)

    # Generate x-axis points
    x_axis = np.linspace(0.01, 5, 200)

    ## Far field calculation
    P_far_axis = 0.5 * c * U0 * k * a * a / x_axis
    I_far_axis = 10 * np.log10(np.square(P_far_axis))

    ## Full field calculation
    P_full_axis = rho0 * c * U0 * (np.exp(-1j * k * x_axis) - np.exp(-1j * k * np.sqrt(np.square(x_axis) + np.square(a))))
    I_full_axis = 10 * np.log10(np.abs(P_full_axis)**2)

    return x_axis, I_far_axis, I_full_axis

def update_far_and_full_field_intensity_on_axis(f, a, SL):
    x_axis, I_far_axis, I_full_axis = far_and_full_field_intensity_on_axis(f, a, SL)
    
    # Close any existing plots
    plt.close('all')

    # Plotting
    plt.figure(1)
    plt.plot(x_axis, I_far_axis, label='Far-field intensity along main beam axis', linewidth=2, color="b")
    plt.plot(x_axis, I_full_axis, label='Full-field intensity along main beam axis', linewidth=2, color="r")
    plt.xlabel('Range (m)')
    plt.ylabel('Intensity (dB re 1μPa)')
    # plt.yaxis.set_label_position('right')
    # plt.yaxis.label.set_rotation(270)
    # plt.yaxis.labelpad = 13
    # plt.yaxis.tick_right()
    # plt.set_aspect('auto')
    plt.grid(True)
    
    plt.show


# Use interact to link widgets and plotting function
interact(
    update_far_and_full_field_intensity_on_axis,
    f=f_slider,
    a=a_slider,
    SL=SL_slider,
)


interactive(children=(IntSlider(value=12, description='Frequency (kHz)', max=200, min=1), FloatSlider(value=0.…

<function __main__.update_far_and_full_field_intensity_on_axis(f, a, SL)>

:::{admonition} Exercise
For the SS510 transducer, we found that at 200 kHz, the radius of the transducer was about 23 mm by using the [beampattern widget](acoustics-SL_piston_far_field). For this frequency and radius, where is the near-field to far-field transition?
:::

(acoustics-source_level)=
# Source Level

Since we are looking at the intensity along the axis of the main beam, this is a good time to define the **source level (SL)**. To determine the source level, the pressure in the far-field along the axis of the main beam is extrapolated back to a range of 1 m from the transducer. The mean square of the pressure, $p_{RMS}^2$, where $p_{RMS}$ is the root mean square of the pressure, is then taken and referenced to an pressure of $p_{ref} = 1 \mu\text{PA}$. In decibels, the source level is expressed as 

$$ SL = 20\log_{10}\frac{p_{RMS}\left(r_{ref}\right)}{p_{ref}},$$

where $r_{ref} =$ 1 m. In the calculation above, you can change the source level. Notice that regardless of what the full-field expression is doing or where the near-to-far-field transition is located, the far-field value at 1 m is always at the source just as we defined it.

# Source Directivity

How do we account for the beam pattern in the sonar equation? The source level is defined in terms of the intensity along the main beam of the transducer. If we are interested in the intensity away from this axis, we can incorporate the **source directivity (D)** into the sonar equation. The source directivity is defined as the ratio of the intensity measured at a given range and at an angle relative to the main beam axis to the intensity at the same range but along the main beam axis,

$$ D = 10\log_{10}\frac{I\left(r,\theta,\phi\right)}{I_{ax}\left(r\right)} $$

where $I_{ax}$ is the intensity along the main beam axis where $\theta = 0$ and $\phi = 0$. This is what we plotted in the [beampattern widget](acoustics-SL_piston_far_field).

For our orca example, if Ola turns is trying find a fish but is facing the wrong direction, the sonar equation becomes

$$ RL = SL + D - 2TL + TS.$$

Although the source directivity is added to the source level, it is typically (but not always) a negative number as you rotate away from the main beam. Thus if Ola is facing the wrong direction, the return from the fish will be weaker.